In [11]:
from dimod import DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler, LeapHybridSampler
import networkx as nx
from copy import deepcopy

token = 'DEV-22b61bae4e0a57f9322c49cb3206d3dddad69d7a'

scheduleDays = 7
numShifts = 3
numNurses = 4

multipleShiftsCost = 10

### Helper programs
# returns composite index given (shift, day)
def Get_Index(shift, day):
    return shift * scheduleDays + day


# Inverse of get_index - given a composite index in a 1D list, return the
# nurse_index and day_index
def Get_Shift_and_Day(index):
    shift, day = divmod(index, scheduleDays)
    return shift, day

graphSize = numShifts*scheduleDays
nurses = range(numNurses)
shiftList = list(range(numShifts))

dqm = DiscreteQuadraticModel()

# Add nodes with number of options each can have
for node in range(graphSize):
    dqm.add_variable(numNurses, label = node)

# Make list of tuple edges
for node in range(graphSize):
    isDayAfter = False
    otherShifts = deepcopy(shiftList)
    
    shift, day = Get_Shift_and_Day(node)
    otherShifts.remove(shift)
    dayAfter = day + 1

    if day + 1 < scheduleDays:
        isDayAfter = True
        
    # Don't schedule the same person for multiple shifts
    for ii in otherShifts:
        index = Get_Index(ii, day)
        dqm.set_quadratic(node, index, {(n, n): multipleShiftsCost for n in nurses})
            
    # Don't change a nurse's schedule from 1 day to another
       
sampler = LeapHybridDQMSampler(endpoint='https://cloud.dwavesys.com/sapi/', token=token, solver={ 'name': 'hybrid_discrete_quadratic_model_version1'})

sampleset = sampler.sample_dqm(dqm)
sample = sampleset.first.sample
energy = sampleset.first.energy
valid = True
'''
for thisDay in range(scheduleDays):
    
    if sample[i] == sample[j]:
        valid = False
        break
'''
# print("Solution: ", sample)
print("Solution energy: ", energy)

for ii in range(numShifts):
    str_row = ""
    for jj in range(scheduleDays):
        thisIndex = Get_Index(ii, jj)
        str_row += str(sample[thisIndex]) + ' '
    print(str_row)

print("Solution validity: ", valid)


Solution energy:  0.0
0 2 0 1 0 2 1 
2 1 2 2 1 3 0 
1 0 3 0 2 1 2 
Solution validity:  True


In [8]:
print(len(sample))

21
